In [1]:
import pandas
import pgreaper
import folium

In [2]:
katrina_path = pandas.read_sql(
    "SELECT * FROM ibtracs_hurricanes "
    "WHERE name LIKE 'KATRINA' "
    "AND season = 2005",
    con=pgreaper.PG_DEFAULTS(dbname='us_wth').to_string()
)
katrina_path

,serial_num,season,num,name,basin,sub_basin,wind_wmo,saffir_simpson,iso_time,latitude_for_mapping,longitude_for_mapping
0,2005236N23285,2005,12,KATRINA,NA,NA,30.0,Tropical Depression,2005-08-23 18:00:00,23.1,-75.1
1,2005236N23285,2005,12,KATRINA,NA,NA,30.0,Tropical Depression,2005-08-24 00:00:00,23.4,-75.7
2,2005236N23285,2005,12,KATRINA,NA,NA,30.0,Tropical Depression,2005-08-24 06:00:00,23.8,-76.2
3,2005236N23285,2005,12,KATRINA,NA,NA,35.0,Tropical Storm,2005-08-24 12:00:00,24.5,-76.5
4,2005236N23285,2005,12,KATRINA,NA,NA,40.0,Tropical Storm,2005-08-24 18:00:00,25.4,-76.9
5,2005236N23285,2005,12,KATRINA,NA,NA,45.0,Tropical Storm,2005-08-25 00:00:00,26.0,-77.7
6,2005236N23285,2005,12,KATRINA,NA,NA,50.0,Tropical Storm,2005-08-25 06:00:00,26.1,-78.4
7,2005236N23285,2005,12,KATRINA,NA,NA,55.0,Tropical Storm,2005-08-25 12:00:00,26.2,-79.0
8,2005236N23285,2005,12,KATRINA,NA,NA,60.0,Tropical Storm,2005-08-25 18:00:00,26.2,-79.6
9,2005236N23285,2005,12,KATRINA,NA,NA,70.0,Category 1,2005-08-25 22:30:00,26.0,-80.1


In [3]:
katrina_path['latitude_for_mapping']

0     23.1
1     23.4
2     23.8
3     24.5
4     25.4
5     26.0
6     26.1
7     26.2
8     26.2
9     26.0
10    25.9
11    25.4
12    25.1
13    24.9
14    24.6
15    24.4
16    24.4
17    24.5
18    24.8
19    25.2
20    25.7
21    26.3
22    27.2
23    28.2
24    29.3
25    29.5
26    30.2
27    31.1
28    32.6
29    34.1
30    35.6
31    37.0
32    38.6
33    40.1
Name: latitude_for_mapping, dtype: float64

Map Tiles
https://deparkes.co.uk/2016/06/10/folium-map-tiles/

In [4]:
def sshs_color(sshs):
    # Colors from Wikipedia
    colors = {
        'Tropical Depression': '#5ebaff',
        'Tropical Storm': '#00faf4',
        'Category 1': '#ffffcc',
        'Category 2': '#ffe775',
        'Category 3': '#ffc140',
        'Category 4': '#ff8f20',
        'Category 5': '#ff6060'        
    }
    
    return colors[sshs]

def sshs_opacity(sshs):
    colors = {
        'Tropical Depression': 0.4,
        'Tropical Storm': 0.5,
        'Category 1': 0.6,
        'Category 2': 0.7,
        'Category 3': 0.8,
        'Category 4': 0.9,
        'Category 5': 1      
    }
    
    return colors[sshs]

In [5]:
help(folium.RegularPolygonMarker)

Help on class RegularPolygonMarker in module folium.features:

class RegularPolygonMarker(folium.map.Marker)
 |  Custom markers using the Leaflet Data Vis Framework.
 |  
 |  Parameters
 |  ----------
 |  location: tuple or list, default None
 |      Latitude and Longitude of Marker (Northing, Easting)
 |  color: string, default 'black'
 |      Marker line color
 |  opacity: float, default 1
 |      Line opacity, scale 0-1
 |  weight: int, default 2
 |      Stroke weight in pixels
 |  fill_color: string, default 'blue'
 |      Marker fill color
 |  fill_opacity: float, default 1
 |      Marker fill opacity
 |  number_of_sides: int, default 4
 |      Number of polygon sides
 |  rotation: int, default 0
 |      Rotation angle in degrees
 |  radius: int, default 15
 |      Marker radius, in pixels
 |  popup: string or folium.Popup, default None
 |      Input text or visualization for object. Can pass either text,
 |      or a folium.Popup object.
 |      If None, no popup will be displaye

In [6]:
help(folium.Popup)

Help on class Popup in module folium.map:

class Popup(branca.element.Element)
 |  Create a Popup instance that can be linked to a Layer.
 |  
 |  Parameters
 |  ----------
 |  html: string or Element
 |      Content of the Popup.
 |  parse_html: bool, default False
 |      True if the popup is a template that needs to the rendered first.
 |  max_width: int, default 300
 |      The maximal width of the popup.
 |  
 |  Method resolution order:
 |      Popup
 |      branca.element.Element
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, html=None, parse_html=False, max_width=300)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  render(self, **kwargs)
 |      Renders the HTML representation of the element.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from branca.element.Element:
 |  
 |  add_child(self, child, name=None, index=None)
 |      Add a child.
 |  
 |  add_childr

In [28]:
def create_popup(sshs, time):
    return folium.Popup(
        html=("<h3>Hurricane Ching Chong</h3>"
              "<h4>{}</h4>"
              "<h5>{}</h5> "
              "<b>Aliases:</b> Amils Huge Weiner").format(sshs, time))

In [29]:
katrina_map = folium.Map(tiles='Stamen Terrain', attr='Stamen Design')

for x, y, sshs, time in zip(katrina_path['latitude_for_mapping'],
             katrina_path['longitude_for_mapping'],
             katrina_path['saffir_simpson'],
             katrina_path['iso_time']):
    folium.RegularPolygonMarker([x, y],
        weight=0,
        number_of_sides=500,
        fill_color=sshs_color(sshs),
        radius=15 * sshs_opacity(sshs),
        popup=create_popup(sshs, time)).add_to(katrina_map)

In [30]:
katrina_map